In [1]:
# Connect to workspace
from azureml.core import Workspace
ws = Workspace.from_config()

In [2]:
# Create Docker environment
from azureml.core import Environment

myenv = Environment(name="myenv")
myenv.docker.enabled = True
with open("Dockerfile", "r") as f:
    dockerfile=f.read()
myenv.docker.base_dockerfile = dockerfile
myenv.python.user_managed_dependencies = True

Property base_image is mutually exclusive with base_dockerfile. Reset base_image to None


In [28]:
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.pipeline.core import PipelineData
from azureml.pipeline.core import Pipeline

# Get the batch dataset for input
batch_data_set = ws.datasets['diabetes']

# Set the output location
default_ds = ws.get_default_datastore()
output_dir = PipelineData(name='inferences',
                          datastore=default_ds,
                          output_path_on_compute='results')

# Define the parallel run step step configuration
parallel_run_config = ParallelRunConfig(
    source_directory='scripts',
    entry_script="batch_pred.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=myenv,
    compute_target='cluster',
    node_count=2)

# Create the parallel run step
parallelrun_step = ParallelRunStep(
    name='batch-score',
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('diabetes')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

# Create the pipeline
pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])

In [29]:
from azureml.core import Experiment

# Run the pipeline as an experiment
pipeline_run = Experiment(ws, 'batch_prediction_pipeline').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

atus
[2020-12-23T13:54:11.392904] Entering context manager injector.
[context_manager_injector.py] Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError'], invocation=['driver/amlbi_main.py', '--client_sdk_version', '1.19.0', '--scoring_module_name', 'batch_pred.py', '--mini_batch_size', '5', '--error_threshold', '10', '--output_action', 'append_row', '--logging_level', 'INFO', '--run_invocation_timeout', '60', '--run_max_try', '3', '--create_snapshot_at_runtime', 'True', '--output', '/mnt/batch/tasks/shared/LS_root/jobs/dscert/azureml/294027c1-3694-454e-8ead-dbeb0fb6d124/mounts/workspaceblobstore/azureml/294027c1-3694-454e-8ead-dbeb0fb6d124/inferences', '--input_ds_0', 'diabetes'])
Script type = None
Starting the daemon thread to refresh tokens in background for process with pid = 141
Entering Run History Context Manager.
[2020-12-23T13:54:15.170333] Curr

'Finished'

In [31]:
# Publish
published_pipeline = pipeline_run.publish_pipeline(name='Batch_Prediction_Pipeline', description='Batch pipeline', version='1.0')
rest_endpoint = published_pipeline.endpoint